# End-to-End LSTM Trajectory Prediction with OpenSky Data

This notebook demonstrates a complete workflow for aircraft trajectory prediction using the LSTMMultiHorizon model.

## Overview

This notebook covers:
1. **Data Loading**: OpenSky historical data or synthetic data generation
2. **Feature Preprocessing**: Process sequences with 8 core features (velocity, track, time-of-day)
3. **Model Training**: Train a multi-layer LSTM model for multi-horizon trajectory prediction
4. **Evaluation**: Assess performance using ADE/FDE metrics and visualizations

### Features Used (8 total)
- **vx, vy, vz**: Velocity components in East, North, Up (m/s)
- **speed**: Horizontal speed (m/s)
- **cos_track, sin_track**: Track angle encoding
- **sin_tod, cos_tod**: Time of day encoding

### Model Output
- Predicts future positions as offsets (Δe, Δn, Δu) in ENU coordinates
- Multi-horizon: predicts K future timesteps from H historical timesteps

## OpenSky Historical Data Integration

This notebook is now integrated with the **OpenSky historical data ingestion pipeline**:

### Using OpenSky Data

**Step 1**: Download historical flight data from OpenSky Network
- Visit: https://opensky-network.org/datasets/states/
- Download state vectors in Parquet format (e.g., `opensky_states_2024-01.parquet`)

**Step 2**: Process the data using the ingestion script
```bash
python ../opensky_historical_ingestion.py --input /path/to/opensky_states.parquet --output data/raw/opensky_data.csv
```

**Step 3**: The script outputs a CSV with columns compatible with our pipeline:
- `flight_id`: Aircraft identifier (ICAO24 code)
- `timestamp`: Unix timestamp (seconds)
- `lat`, `lon`: Geographic coordinates (degrees)
- `alt`: Altitude (meters, filtered to `alt > 0`)

**Step 4**: Load and use the data in this notebook (see Data Loading section below)

### Alternative: Synthetic Data

For quick experimentation without external dependencies, you can generate synthetic data using the built-in data generator.

## 1. Setup and Environment

First, we'll set up the environment by installing dependencies and importing necessary libraries.

In [ ]:
# Install required packages (run once)
# Uncomment the following line if running in Colab or if packages are not installed
# !pip install -q torch numpy pandas matplotlib seaborn tqdm scikit-learn pyproj pymap3d pyyaml requests

In [ ]:
# Import standard libraries
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import time

# Deep learning imports
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

# Set plotting style
sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

print(f"PyTorch version: {torch.__version__}")
print(f"Device: {'CUDA' if torch.cuda.is_available() else 'CPU'}")

In [ ]:
# Add src to path if not already there
import os
src_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))
if src_path not in sys.path:
    sys.path.insert(0, src_path)

print("✓ Directories created")
print("✓ Python path configured")

## 2. Data Loading: OpenSky Historical Data or Synthetic

You can choose between two data sources:

### Option A: OpenSky Historical Data (Recommended for Production)

Use real aircraft trajectory data from the OpenSky Network. This provides production-quality training data with real flight patterns.

**Prerequisites**:
1. Download OpenSky parquet file from https://opensky-network.org/datasets/states/
2. Process it using: `python ../opensky_historical_ingestion.py --input <parquet_file> --output data/raw/opensky_data.csv`

The processed CSV will have the required format:
- **flight_id**: Aircraft ICAO24 identifier
- **timestamp**: Unix timestamp (seconds)
- **lat, lon**: Geographic coordinates (degrees)
- **alt**: Altitude in meters (filtered to positive values)

### Option B: Synthetic Data (Quick Start)

Generate synthetic flight trajectories for immediate experimentation. This is useful for:
- Testing the pipeline without external data
- Quick prototyping and development
- Reproducible experiments

**Data will be generated around Hong Kong International Airport (VHHH)**.

In [ ]:
# ================================================================================
# OPTION A: Load OpenSky Historical Data (uncomment to use)
# ================================================================================

# import os
# 
# # Path to your processed OpenSky CSV file
# # This file should be created by running:
# # python ../opensky_historical_ingestion.py --input <parquet_file> --output data/raw/opensky_data.csv
# OPENSKY_CSV = 'data/raw/opensky_data.csv'
# 
# if os.path.exists(OPENSKY_CSV):
#     print(f"Loading OpenSky historical data from: {OPENSKY_CSV}")
#     df_raw = pd.read_csv(OPENSKY_CSV)
#     
#     print(f"Raw data shape: {df_raw.shape}")
#     print(f"Columns: {list(df_raw.columns)}")
#     print(f"Number of unique flights: {df_raw['flight_id'].nunique()}")
#     print(f"Time span: {pd.to_datetime(df_raw['timestamp'].min(), unit='s')} to {pd.to_datetime(df_raw['timestamp'].max(), unit='s')}")
#     print(f"Altitude range: {df_raw['alt'].min():.0f}m to {df_raw['alt'].max():.0f}m")
#     
#     # Filter to ensure positive altitudes (pipeline requirement)
#     df_raw = df_raw[df_raw['alt'] > 0]
#     print(f"\nAfter altitude filtering (alt > 0): {len(df_raw)} rows")
#     
#     # Display first few rows
#     print("\nFirst few rows:")
#     display(df_raw.head())
#     
#     print("\n✓ OpenSky historical data loaded successfully")
# else:
#     print(f"Error: OpenSky CSV file not found at {OPENSKY_CSV}")
#     print("\nPlease follow these steps:")
#     print("1. Download OpenSky parquet file from: https://opensky-network.org/datasets/states/")
#     print("2. Run: python ../opensky_historical_ingestion.py --input <parquet_file> --output data/raw/opensky_data.csv")
#     print("3. Re-run this cell")
#     raise FileNotFoundError(f"OpenSky CSV not found: {OPENSKY_CSV}")

# ================================================================================
# OPTION B: Generate Synthetic Data (default)
# ================================================================================

# Option B: Generate synthetic data (recommended for demo)
# This creates realistic synthetic flights around Hong Kong (VHHH)

print("Generating synthetic Hong Kong airspace data...")
!python ../src/simulate_data.py \
    --num_flights 600 \
    --min_len 160 \
    --max_len 260 \
    --out_csv data/raw/hkg_data.csv

print("\n✓ Synthetic data generated")

In [ ]:
# Note: The following cells work with BOTH OpenSky historical data and synthetic data
# because both produce CSV files with the same format:
# flight_id, timestamp, lat, lon, alt

# If using OpenSky data (Option A above), change the file path:
# df_raw = pd.read_csv('data/raw/opensky_data.csv')  # For OpenSky data

# If using synthetic data (Option B - default):
df_raw = pd.read_csv('data/raw/hkg_data.csv')  # For synthetic data

# The rest of the pipeline works identically for both data sources
print(f"Raw data shape: {df_raw.shape}")
print(f"\nColumns: {list(df_raw.columns)}")
print(f"\nNumber of unique flights: {df_raw['flight_id'].nunique()}")
print(f"Time span: {df_raw['timestamp'].min()} to {df_raw['timestamp'].max()}")
print(f"Altitude range: {df_raw['alt'].min():.0f}m to {df_raw['alt'].max():.0f}m")

# Display first few rows
print("\nFirst few rows:")
df_raw.head()

In [ ]:
# Load and inspect the raw data
df_raw = pd.read_csv('data/raw/hkg_data.csv')

print(f"Raw data shape: {df_raw.shape}")
print(f"\nColumns: {list(df_raw.columns)}")
print(f"\nNumber of unique flights: {df_raw['flight_id'].nunique()}")
print(f"Time span: {df_raw['timestamp'].min()} to {df_raw['timestamp'].max()}")
print(f"Altitude range: {df_raw['alt'].min():.0f}m to {df_raw['alt'].max():.0f}m")

# Display first few rows
print("\nFirst few rows:")
df_raw.head()

## 3. Feature Preprocessing

Now we'll preprocess the data to create input sequences for the LSTM model:

### Steps:
1. **Region Filtering**: Keep only flights within Hong Kong airspace
2. **Temporal Resampling**: Standardize time intervals (e.g., 30 seconds)
3. **Coordinate Transformation**: Convert lat/lon/alt to local ENU (East-North-Up) coordinates
4. **Feature Engineering**: Compute velocity, speed, track angle, and time-of-day features
5. **Sequence Windowing**: Create (history, future) pairs for training

### Parameters:
- **input_len** (H): Number of historical timesteps (e.g., 40)
- **pred_len** (K): Number of future timesteps to predict (e.g., 20)
- **resample_sec**: Time resolution in seconds (e.g., 30s)

In [ ]:
# Run the preprocessing pipeline
# This creates sequences suitable for LSTM training

INPUT_LEN = 40  # History window (H)
PRED_LEN = 20   # Prediction horizon (K)
RESAMPLE_SEC = 30  # Time resolution
REGION = "113.5,115.5,21.5,23.0"  # Hong Kong region (lon_min,lon_max,lat_min,lat_max)

print("Preprocessing data into sequences...")
!python ../src/prepare_sequences.py \
    --input_csv data/raw/hkg_data.csv \
    --output_npz data/processed/hkg_seq.npz \
    --input_len {INPUT_LEN} \
    --pred_len {PRED_LEN} \
    --resample_sec {RESAMPLE_SEC} \
    --region "{REGION}" \
    --min_points 120

print("\n✓ Preprocessing complete")

In [ ]:
# Load and inspect the preprocessed data
data = np.load('data/processed/hkg_seq.npz', allow_pickle=True)

X = data['X']  # Input sequences: [N, H, F] where F=8 features
Y = data['Y']  # Target sequences: [N, K, 3] where 3=(Δe, Δn, Δu)
flights = data['flights']  # Flight IDs for each sequence

print(f"Input sequences (X): {X.shape}")
print(f"  - N: {X.shape[0]} sequences")
print(f"  - H: {X.shape[1]} timesteps (history)")
print(f"  - F: {X.shape[2]} features")
print(f"\nTarget sequences (Y): {Y.shape}")
print(f"  - N: {Y.shape[0]} sequences")
print(f"  - K: {Y.shape[1]} timesteps (future)")
print(f"  - 3: (east, north, up) offsets in meters")
print(f"\nUnique flights: {len(np.unique(flights))}")

# Feature names for reference
feature_names = ['vx', 'vy', 'vz', 'speed', 'cos_track', 'sin_track', 'sin_tod', 'cos_tod']
print(f"\nFeatures: {feature_names}")

In [ ]:
# Visualize sample trajectories in ENU coordinates
# Plot a few examples to understand the data

n_samples = 4
idx = np.random.choice(len(Y), size=n_samples, replace=False)

fig, axes = plt.subplots(1, n_samples, figsize=(16, 4))
for i, ax in enumerate(axes):
    y_sample = Y[idx[i]]  # [K, 3]
    # Plot future trajectory offsets
    ax.plot(y_sample[:, 0], y_sample[:, 1], 'o-', linewidth=2, markersize=4)
    ax.plot([0], [0], 'r*', markersize=15, label='Start (last history point)')
    ax.axhline(0, color='gray', linewidth=0.5, linestyle='--')
    ax.axvline(0, color='gray', linewidth=0.5, linestyle='--')
    ax.set_xlabel('East offset (m)')
    ax.set_ylabel('North offset (m)')
    ax.set_title(f'Sample {idx[i]}')
    ax.grid(True, alpha=0.3)
    if i == 0:
        ax.legend()

plt.tight_layout()
plt.savefig('outputs/notebook_run/sample_trajectories.png', dpi=120, bbox_inches='tight')
plt.show()

print("✓ Sample trajectories saved to outputs/notebook_run/sample_trajectories.png")

## 4. Model Architecture: LSTMMultiHorizon

Our model uses a multi-layer LSTM network for sequence-to-sequence prediction:

### Architecture:
```
Input: [Batch, H, F] where F=8 features
  ↓
LSTM Layers (with dropout)
  ↓
Take last hidden state: [Batch, hidden_size]
  ↓
FC Layer → ReLU → Dropout → FC Layer
  ↓
Output: [Batch, K, 3] where 3=(Δe, Δn, Δu)
```

### Hyperparameters:
- **hidden_size**: LSTM hidden dimension (e.g., 128)
- **num_layers**: Number of stacked LSTM layers (e.g., 2)
- **dropout**: Dropout rate for regularization (e.g., 0.2)

Let's import and inspect the model:

In [ ]:
# Import model from src
try:
    from model import LSTMMultiHorizon
except ImportError:
    # If direct import fails, try with explicit src prefix
    import sys
    import os
    src_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))
    if src_path not in sys.path:
        sys.path.insert(0, src_path)
    from model import LSTMMultiHorizon

# Create model instance
INPUT_SIZE = 8  # Number of features
HIDDEN_SIZE = 128
NUM_LAYERS = 2
DROPOUT = 0.2

model = LSTMMultiHorizon(
    input_size=INPUT_SIZE,
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
    dropout=DROPOUT,
    pred_len=PRED_LEN
)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print("Model Architecture:")
print(model)
print(f"\nTotal parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

In [ ]:
# Test model with dummy input to verify output shape
dummy_input = torch.randn(4, INPUT_LEN, INPUT_SIZE)  # [Batch=4, H=40, F=8]
dummy_output = model(dummy_input)

print(f"Input shape:  {dummy_input.shape}  # [Batch, History, Features]")
print(f"Output shape: {dummy_output.shape}  # [Batch, Prediction, 3]")
print("\n✓ Model forward pass successful")

## 5. Model Training

We'll train the model using:
- **Loss Function**: Smooth L1 Loss (Huber loss) - robust to outliers
- **Optimizer**: AdamW with weight decay for regularization
- **Validation Metrics**: ADE (Average Displacement Error) and FDE (Final Displacement Error)
- **Early Stopping**: Stop when validation ADE stops improving

### Training Process:
1. Split data into train/val/test sets (by flight ID to avoid leakage)
2. Train for multiple epochs
3. Validate after each epoch
4. Save best model based on validation ADE
5. Plot training curves

In [ ]:
# Import training utilities
from dataset import TrajSeqDataset

# Set random seeds for reproducibility
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)

# Create datasets (split by flights to avoid leakage)
ds_train = TrajSeqDataset('data/processed/hkg_seq.npz', split='train', seed=SEED)
ds_val = TrajSeqDataset('data/processed/hkg_seq.npz', split='val', seed=SEED)
ds_test = TrajSeqDataset('data/processed/hkg_seq.npz', split='test', seed=SEED)

print(f"Train set: {len(ds_train)} sequences")
print(f"Val set:   {len(ds_val)} sequences")
print(f"Test set:  {len(ds_test)} sequences")
print(f"\nTotal:     {len(ds_train) + len(ds_val) + len(ds_test)} sequences")

In [ ]:
# Create data loaders
BATCH_SIZE = 128

train_loader = DataLoader(
    ds_train,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

val_loader = DataLoader(
    ds_val,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

print(f"Batches per epoch: {len(train_loader)} (train), {len(val_loader)} (val)")

In [ ]:
# Initialize model, optimizer, and loss function
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

model = LSTMMultiHorizon(
    input_size=INPUT_SIZE,
    hidden_size=HIDDEN_SIZE,
    num_layers=NUM_LAYERS,
    dropout=DROPOUT,
    pred_len=PRED_LEN
).to(device)

# Optimizer and loss
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 0.0
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
criterion = nn.SmoothL1Loss()

print("✓ Model, optimizer, and criterion initialized")

In [ ]:
# Define ADE and FDE computation
def compute_ade_fde(pred, target):
    """
    Compute Average Displacement Error and Final Displacement Error.
    
    Args:
        pred: [B, K, 3] predicted offsets in meters
        target: [B, K, 3] ground truth offsets in meters
    
    Returns:
        ade: Average displacement error (mean over all timesteps)
        fde: Final displacement error (at last timestep)
    """
    diff = pred - target
    dist = torch.linalg.norm(diff, dim=-1)  # [B, K]
    ade = dist.mean().item()
    fde = dist[:, -1].mean().item()
    return ade, fde

print("✓ Metrics defined")

In [ ]:
# Training loop
EPOCHS = 20
GRAD_CLIP = 1.0
PATIENCE_LIMIT = 5

best_val_ade = float('inf')
patience = 0
history = {
    'train_loss': [],
    'val_loss': [],
    'val_ade': [],
    'val_fde': []
}

print("Starting training...\n")
print("="*60)

for epoch in range(1, EPOCHS + 1):
    # ========== Training Phase ==========
    model.train()
    train_loss = 0.0
    
    for xb, yb in tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS} [Train]"):
        xb = xb.to(device)
        yb = yb.to(device)
        
        # Forward pass
        pred = model(xb)
        loss = criterion(pred, yb)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), GRAD_CLIP)
        optimizer.step()
        
        train_loss += loss.item()
    
    train_loss /= len(train_loader)
    
    # ========== Validation Phase ==========
    model.eval()
    val_loss = 0.0
    all_ade, all_fde = [], []
    
    with torch.no_grad():
        for xb, yb in tqdm(val_loader, desc=f"Epoch {epoch}/{EPOCHS} [Val]  "):
            xb = xb.to(device)
            yb = yb.to(device)
            
            pred = model(xb)
            loss = criterion(pred, yb)
            val_loss += loss.item()
            
            # Compute metrics
            ade, fde = compute_ade_fde(pred, yb)
            all_ade.append(ade)
            all_fde.append(fde)
    
    val_loss /= len(val_loader)
    val_ade = float(np.mean(all_ade))
    val_fde = float(np.mean(all_fde))
    
    # Record history
    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['val_ade'].append(val_ade)
    history['val_fde'].append(val_fde)
    
    # Print epoch summary
    print(f"Epoch {epoch:2d}/{EPOCHS}: "
          f"train_loss={train_loss:.4f}, "
          f"val_loss={val_loss:.4f}, "
          f"val_ADE={val_ade:.2f}m, "
          f"val_FDE={val_fde:.2f}m")
    
    # ========== Early Stopping & Model Saving ==========
    if val_ade < best_val_ade - 1e-3:
        best_val_ade = val_ade
        patience = 0
        
        # Save best model
        torch.save(model.state_dict(), 'outputs/notebook_run/best/model.pt')
        with open('outputs/notebook_run/best/meta.txt', 'w') as f:
            f.write(f"epoch={epoch}\nval_ADE={val_ade}\nval_FDE={val_fde}\n")
        print(f"  → New best model saved (val_ADE: {val_ade:.2f}m)")
    else:
        patience += 1
        if patience >= PATIENCE_LIMIT:
            print(f"\nEarly stopping triggered (patience={PATIENCE_LIMIT})")
            break
    
    print("="*60)

print("\n✓ Training complete!")
print(f"Best validation ADE: {best_val_ade:.2f}m")

In [ ]:
# Plot training curves
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss curves
axes[0].plot(history['train_loss'], label='Train Loss', linewidth=2)
axes[0].plot(history['val_loss'], label='Val Loss', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=12)
axes[0].set_ylabel('Smooth L1 Loss', fontsize=12)
axes[0].set_title('Training and Validation Loss', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# Metrics curves
axes[1].plot(history['val_ade'], label='Val ADE (m)', linewidth=2, marker='o')
axes[1].plot(history['val_fde'], label='Val FDE (m)', linewidth=2, marker='s')
axes[1].set_xlabel('Epoch', fontsize=12)
axes[1].set_ylabel('Error (meters)', fontsize=12)
axes[1].set_title('Validation Metrics', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('outputs/notebook_run/training_curves.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Training curves saved to outputs/notebook_run/training_curves.png")

## 6. Model Evaluation

Now let's evaluate the trained model on the test set and compare it with a constant velocity baseline.

### Evaluation Metrics:
- **ADE (Average Displacement Error)**: Mean Euclidean distance between predicted and true positions across all timesteps
- **FDE (Final Displacement Error)**: Euclidean distance at the final timestep only

### Baseline:
We'll compare against a simple constant velocity model that extrapolates using the last 5 velocity measurements.

In [ ]:
# Load best model
model.load_state_dict(torch.load('outputs/notebook_run/best/model.pt', map_location=device))
model.eval()
print("✓ Best model loaded")

In [ ]:
# Create test data loader
test_loader = DataLoader(
    ds_test,
    batch_size=256,
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

# Evaluate on test set
all_pred, all_true = [], []

print("Evaluating on test set...")
with torch.no_grad():
    for xb, yb in tqdm(test_loader, desc="Test"):
        xb = xb.to(device)
        pred = model(xb).cpu().numpy()
        all_pred.append(pred)
        all_true.append(yb.numpy())

pred_test = np.concatenate(all_pred, axis=0)
true_test = np.concatenate(all_true, axis=0)

print(f"\nTest predictions shape: {pred_test.shape}")
print(f"Test ground truth shape: {true_test.shape}")

In [ ]:
# Compute test metrics
def compute_ade_fde_np(pred, true):
    """Compute ADE/FDE using NumPy."""
    diff = pred - true
    dist = np.linalg.norm(diff, axis=-1)  # [N, K]
    ade = float(dist.mean())
    fde = float(dist[:, -1].mean())
    return ade, fde

test_ade, test_fde = compute_ade_fde_np(pred_test, true_test)

print("="*60)
print("MODEL PERFORMANCE ON TEST SET")
print("="*60)
print(f"ADE (Average Displacement Error): {test_ade:.2f} meters")
print(f"FDE (Final Displacement Error):   {test_fde:.2f} meters")
print("="*60)

In [ ]:
# Constant velocity baseline
def constant_velocity_baseline(X_hist, pred_len, dt=30):
    """
    Simple baseline: extrapolate using mean velocity from last 5 timesteps.
    
    Args:
        X_hist: [H, F] with vx, vy, vz as features 0, 1, 2
        pred_len: K, number of future steps
        dt: time resolution in seconds
    
    Returns:
        offsets: [K, 3] predicted offsets
    """
    vx = X_hist[-5:, 0].mean()
    vy = X_hist[-5:, 1].mean()
    vz = X_hist[-5:, 2].mean()
    
    steps = np.arange(1, pred_len + 1, dtype=np.float32).reshape(-1, 1)
    offsets = np.hstack([vx * steps * dt, vy * steps * dt, vz * steps * dt])
    return offsets

# Compute baseline on a subset (for efficiency)
print("Computing constant velocity baseline...")
n_baseline = min(2000, len(ds_test))
idx_baseline = np.random.choice(len(ds_test), size=n_baseline, replace=False)

Xs_baseline = ds_test.X[idx_baseline]
Ys_baseline = ds_test.Y[idx_baseline]

baseline_pred = np.stack(
    [constant_velocity_baseline(Xs_baseline[i], PRED_LEN, RESAMPLE_SEC) 
     for i in tqdm(range(n_baseline), desc="Baseline")],
    axis=0
)

baseline_ade, baseline_fde = compute_ade_fde_np(baseline_pred, Ys_baseline)

print("\n" + "="*60)
print("BASELINE PERFORMANCE")
print("="*60)
print(f"ADE: {baseline_ade:.2f} meters")
print(f"FDE: {baseline_fde:.2f} meters")
print("="*60)

In [ ]:
# Summary comparison
print("\n" + "="*60)
print("COMPARISON: MODEL vs BASELINE")
print("="*60)
print(f"{'Metric':<20} {'Model':<15} {'Baseline':<15} {'Improvement':<15}")
print("-"*60)
print(f"{'ADE (meters)':<20} {test_ade:<15.2f} {baseline_ade:<15.2f} {(1 - test_ade/baseline_ade)*100:>13.1f}%")
print(f"{'FDE (meters)':<20} {test_fde:<15.2f} {baseline_fde:<15.2f} {(1 - test_fde/baseline_fde)*100:>13.1f}%")
print("="*60)

# Save results
with open('outputs/notebook_run/test_metrics.txt', 'w') as f:
    f.write(f"Model ADE: {test_ade:.3f}\n")
    f.write(f"Model FDE: {test_fde:.3f}\n")
    f.write(f"Baseline ADE: {baseline_ade:.3f}\n")
    f.write(f"Baseline FDE: {baseline_fde:.3f}\n")

print("\n✓ Test metrics saved to outputs/notebook_run/test_metrics.txt")

## 7. Visualization: Predictions vs Ground Truth

Let's visualize some sample predictions to qualitatively assess the model's performance.

In [ ]:
# Plot sample trajectory predictions
n_samples = 6
sample_idx = np.random.choice(len(true_test), size=n_samples, replace=False)

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, idx in enumerate(sample_idx):
    ax = axes[i]
    
    # Get true and predicted trajectories
    true_traj = true_test[idx]  # [K, 3]
    pred_traj = pred_test[idx]  # [K, 3]
    
    # Plot in 2D (East-North plane)
    ax.plot(true_traj[:, 0], true_traj[:, 1], 'o-', 
            label='Ground Truth', linewidth=2.5, markersize=6, alpha=0.8)
    ax.plot(pred_traj[:, 0], pred_traj[:, 1], 's-', 
            label='LSTM Prediction', linewidth=2.5, markersize=6, alpha=0.8)
    
    # Mark start point
    ax.plot([0], [0], 'r*', markersize=20, label='Start', zorder=10)
    
    # Reference lines
    ax.axhline(0, color='gray', linewidth=0.8, linestyle='--', alpha=0.5)
    ax.axvline(0, color='gray', linewidth=0.8, linestyle='--', alpha=0.5)
    
    # Compute error for this sample
    sample_error = np.linalg.norm(true_traj - pred_traj, axis=-1).mean()
    
    ax.set_xlabel('East Offset (m)', fontsize=11)
    ax.set_ylabel('North Offset (m)', fontsize=11)
    ax.set_title(f'Sample {idx} (ADE: {sample_error:.1f}m)', fontsize=12, fontweight='bold')
    ax.grid(True, alpha=0.3)
    ax.legend(fontsize=9, loc='best')
    ax.set_aspect('equal', adjustable='box')

plt.tight_layout()
plt.savefig('outputs/notebook_run/predictions_vs_groundtruth.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Prediction visualizations saved to outputs/notebook_run/predictions_vs_groundtruth.png")

In [ ]:
# Error distribution histogram
# Compute per-timestep errors across all test samples
errors = np.linalg.norm(pred_test - true_test, axis=-1)  # [N, K]
errors_flat = errors.flatten()

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Histogram of all errors
axes[0].hist(errors_flat, bins=50, edgecolor='black', alpha=0.7)
axes[0].axvline(test_ade, color='red', linewidth=2, linestyle='--', label=f'Mean ADE: {test_ade:.2f}m')
axes[0].set_xlabel('Displacement Error (m)', fontsize=12)
axes[0].set_ylabel('Frequency', fontsize=12)
axes[0].set_title('Distribution of Displacement Errors', fontsize=14, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)

# Error vs timestep
mean_errors_per_step = errors.mean(axis=0)  # [K]
std_errors_per_step = errors.std(axis=0)    # [K]
timesteps = np.arange(1, PRED_LEN + 1)

axes[1].plot(timesteps, mean_errors_per_step, 'o-', linewidth=2, markersize=6, label='Mean Error')
axes[1].fill_between(timesteps, 
                      mean_errors_per_step - std_errors_per_step,
                      mean_errors_per_step + std_errors_per_step,
                      alpha=0.3, label='±1 Std Dev')
axes[1].set_xlabel('Prediction Timestep', fontsize=12)
axes[1].set_ylabel('Displacement Error (m)', fontsize=12)
axes[1].set_title('Error Growth Over Prediction Horizon', fontsize=14, fontweight='bold')
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('outputs/notebook_run/error_analysis.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ Error analysis saved to outputs/notebook_run/error_analysis.png")

## 8. 3D Trajectory Visualization

Let's also visualize trajectories in 3D to see vertical movement (altitude changes).

In [ ]:
# 3D trajectory visualization
from mpl_toolkits.mplot3d import Axes3D

n_samples_3d = 3
sample_idx_3d = np.random.choice(len(true_test), size=n_samples_3d, replace=False)

fig = plt.figure(figsize=(16, 5))

for i, idx in enumerate(sample_idx_3d):
    ax = fig.add_subplot(1, 3, i+1, projection='3d')
    
    true_traj = true_test[idx]
    pred_traj = pred_test[idx]
    
    # Plot trajectories
    ax.plot(true_traj[:, 0], true_traj[:, 1], true_traj[:, 2], 
            'o-', label='Ground Truth', linewidth=2, markersize=5, alpha=0.8)
    ax.plot(pred_traj[:, 0], pred_traj[:, 1], pred_traj[:, 2], 
            's-', label='Prediction', linewidth=2, markersize=5, alpha=0.8)
    
    # Start point
    ax.scatter([0], [0], [0], c='red', s=200, marker='*', label='Start', zorder=10)
    
    ax.set_xlabel('East (m)', fontsize=10)
    ax.set_ylabel('North (m)', fontsize=10)
    ax.set_zlabel('Up (m)', fontsize=10)
    ax.set_title(f'3D Trajectory - Sample {idx}', fontsize=12, fontweight='bold')
    ax.legend(fontsize=9)

plt.tight_layout()
plt.savefig('outputs/notebook_run/3d_trajectories.png', dpi=150, bbox_inches='tight')
plt.show()

print("✓ 3D trajectory visualizations saved to outputs/notebook_run/3d_trajectories.png")

## 9. Summary and Saved Outputs

### What We Accomplished:
1. ✓ Fetched/generated aircraft trajectory data
2. ✓ Preprocessed data into sequences with 8 features
3. ✓ Trained a multi-layer LSTM model
4. ✓ Evaluated with ADE/FDE metrics
5. ✓ Visualized predictions vs ground truth
6. ✓ Saved all results and model checkpoints

### Saved Files:
- `data/raw/hkg_data.csv` - Raw trajectory data
- `data/processed/hkg_seq.npz` - Preprocessed sequences
- `outputs/notebook_run/best/model.pt` - Best model weights
- `outputs/notebook_run/training_curves.png` - Training/validation curves
- `outputs/notebook_run/test_metrics.txt` - Final test metrics
- `outputs/notebook_run/predictions_vs_groundtruth.png` - Prediction visualizations
- `outputs/notebook_run/error_analysis.png` - Error distribution and growth
- `outputs/notebook_run/3d_trajectories.png` - 3D trajectory plots

### Next Steps:
- **Use Real Data**: Replace synthetic data with OpenSky API data
- **Add Weather Features**: Incorporate wind (u/v) from GFS/ERA5
- **Hyperparameter Tuning**: Experiment with different architectures
- **Ensemble Methods**: Combine multiple models
- **Deploy**: Create inference API for real-time predictions

In [ ]:
# Final summary printout
print("="*70)
print(" "*15 + "END-TO-END LSTM TRAJECTORY PREDICTION SUMMARY")
print("="*70)
print(f"\n📊 DATASET STATISTICS:")
print(f"  Total sequences:      {len(ds_train) + len(ds_val) + len(ds_test):,}")
print(f"  Train sequences:      {len(ds_train):,}")
print(f"  Validation sequences: {len(ds_val):,}")
print(f"  Test sequences:       {len(ds_test):,}")
print(f"  History length (H):   {INPUT_LEN} timesteps")
print(f"  Prediction length (K): {PRED_LEN} timesteps")
print(f"  Features:             {INPUT_SIZE}")

print(f"\n🧠 MODEL ARCHITECTURE:")
print(f"  Type:                 LSTMMultiHorizon")
print(f"  Hidden size:          {HIDDEN_SIZE}")
print(f"  LSTM layers:          {NUM_LAYERS}")
print(f"  Dropout:              {DROPOUT}")
print(f"  Total parameters:     {trainable_params:,}")

print(f"\n📈 TRAINING:")
print(f"  Epochs completed:     {len(history['train_loss'])}")
print(f"  Batch size:           {BATCH_SIZE}")
print(f"  Learning rate:        {LEARNING_RATE}")
print(f"  Best val ADE:         {best_val_ade:.2f}m")

print(f"\n🎯 TEST RESULTS:")
print(f"  Model ADE:            {test_ade:.2f} meters")
print(f"  Model FDE:            {test_fde:.2f} meters")
print(f"  Baseline ADE:         {baseline_ade:.2f} meters")
print(f"  Baseline FDE:         {baseline_fde:.2f} meters")
print(f"  Improvement (ADE):    {(1 - test_ade/baseline_ade)*100:.1f}%")
print(f"  Improvement (FDE):    {(1 - test_fde/baseline_fde)*100:.1f}%")

print(f"\n💾 SAVED OUTPUTS:")
saved_files = [
    'data/raw/hkg_data.csv',
    'data/processed/hkg_seq.npz',
    'outputs/notebook_run/best/model.pt',
    'outputs/notebook_run/training_curves.png',
    'outputs/notebook_run/test_metrics.txt',
    'outputs/notebook_run/predictions_vs_groundtruth.png',
    'outputs/notebook_run/error_analysis.png',
    'outputs/notebook_run/3d_trajectories.png'
]
for f in saved_files:
    print(f"  ✓ {f}")

print("\n" + "="*70)
print(" "*20 + "🎉 ALL STEPS COMPLETED SUCCESSFULLY! 🎉")
print("="*70)